# Mini projet de srapping

In [ ]:
'''
-objectif scraper le nombre de votes de chaque film
-exploration de la page html  à scraper
url_film="https://www.imdb.com/search/title/?release_date=2017&sort=num_votes,desc&page=1"
-inspection de la page
-cherche des divet classe contenant des info à scraper
-utilisation des librairies " request" pour la connexion à la page
-utilisation de la librairie "bs4" pour la classe BeautifulSoup 
-<div id="seven-day-forecast" class="panel panel-default">, div des tous les elements  html
- <id=tombstone-container>, div de données
    

'''

In [ ]:
# script final

#suejet: scraper les données du site
'''
etapes à suivres
-declaration des listes qui vont contenir chaque données a scraper(titre,year,metascore,note,vote,genre de film ,durée)
-preparation de l'écran d'afficahge de la boucle (temps et nombre de request)
-ecrire une boucle qui fait varier le parametre release_date de l'url avec les valeurs de la liste year_url
-faire une requete GET sur la boucle  de la liste pages
-faire une pause une chaque  boucle sur un intervalle de 8 à 15 sécondes
-afficher à l'écran le temps de réquetes vu à la vidéos précédente
-ajouter un avertisssment pour tout code status différent de 200
-arreter la boucle si le nombre de requtes est supérieur au nombre attendu (72 pages)
-convertir le contenu HTML response en un BeautifulSoup
-extraire tous les containers des différents films
-écrire une boucle qui parcourt tous les containers
-extraire les informations de chaque conteainer si cellui-ci a un metascore
'''

In [24]:
#1 declaration des listes de données
# importationd es librairie
 
 
# lirairie pour la connexion parser html de la page
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd  
from  time  import sleep
from time import time
from random import randint
from warnings import warn
from IPython.display import clear_output 


'''

scrapingFilms: fonction de scraping 
               input :none

output: 
   dateframe: titre,year,metascore,note,vote,genre de film ,durée
''''

def scrapingFilms():
   
    titre_movie=[]
    year_movie=[]
    vote_movie=[]
    note_movie=[]
    metascore_movie=[]
    genre_movie=[]
    time_movie=[]
    start_runtime=time() 
    nbr_requests=0


    #preparation des liens a inserer sur le url de chaque page
    start_page=1
    end_page=3
    pages=[str(i) for i in range(start_page,end_page)]
    start_year=2008
    end_year=2010
    years_url=[str(j) for j in range(start_year,end_year)]

    # 2-pour chaque année de l'inetrvalle 2000-2018
    for year_url in years_url:
      #3- boucle pour chque page de 1 à 4
      for page in pages:
        # 4-faire une requete GET
        response=requests.get('https://www.imdb.com/search/title/?release_date='+ year_url + '&sort=num_votes,desc&page='+ page)
        
        #5- faire une pause de 8 à 15 secondes
        sleep(randint(8,15))
        # 6-info sur la  requete en cours
        nbr_requests +=1
        last_runtime=time()-start_runtime
        print(f"requeste: {nbr_requests}, frequency: {nbr_requests/last_runtime} request/s")

        #7 affichage du film en cours
        clear_output(wait=True)

        #8- test la bonne connexion
        if response.status_code !=200:
          warn(f"resquets: {nbr_requests} , code_statut:{response.code_status}")


        # 9- condition d'arret si le nombre de page est supérieur à 20
        if  nbr_requests>20:
          warn("attention nombre de requets trop importants")
          break
        
        # 10-extraire les elment html avec  beautifulsoup
        soup_movies=bs(response.text,"html.parser")

        #1-extarire les 50 films de chaque page
        movies_container=soup_movies.find_all('div',class_="lister-item mode-advanced")

        # boucle pour chaque container
        for movie in movies_container:
          if movie.find("div",class_="ratings-metascore") is not None:
              titre_mv=movie.h3.a.text
              titre_movie.append(titre_mv)

              year_mv=movie.h3.find('span',class_="lister-item-year text-muted unbold").get_text()
              year_movie.append(year_mv[1:5])

              note_mv=float(movie.strong.text)
              note_movie.append(note_mv)

              nb_metascore_mv=movie.find('span',class_="metascore").text
              metascore_movie.append(int(nb_metascore_mv))

              nb_votes_mv=movie.find('span',attrs={"name":"nv"})['data-value']
              vote_movie.append(int(nb_votes_mv))

              genre_mv=movie.find('span',class_="genre").get_text()
              genre_movie.append( genre_mv[2:])

              duree_mv=str(movie.find('span',class_="runtime").get_text())
              time_movie.append(duree_mv)
     
      return pd.DataFrame({"titre":titre_movie,"out_year":year_movie,"note":note_movie,"nb_metascore":metascore_movie,"nb_vote":vote_movie,"genre_film":genre_movie,"runtime":time_movie})
      


In [25]:
#appel de la fonction scrapping
res=scrapingFilms()

requeste: 2, frequency: 0.0689074487413958 request/s


In [30]:
res

,titre,out_year,note,nb_metascore,nb_vote,genre_film,runtime
0,The Dark Knight,2008,9.0,84,2483778,"ction, Crime, Drama",152 min
1,WALL·E,2008,8.4,95,1064818,"nimation, Adventure, Family",98 min
2,Iron Man,2008,7.9,79,1009337,"ction, Adventure, Sci-Fi",126 min
3,Slumdog Millionaire,2008,8.0,84,826550,"rama, Romance",120 min
4,Gran Torino,2008,8.1,72,752994,rama,116 min
...,...,...,...,...,...,...,...
87,Revolutionary Road,2008,7.3,69,208117,"rama, Romance",119 min
88,The Boy in the Striped Pyjamas,2008,7.8,55,207854,"rama, History, War",94 min
89,Death Race,2008,6.3,43,205383,"ction, Sci-Fi, Thriller",105 min
90,The Chronicles of Narnia: Prince Caspian,2008,6.5,62,204877,"ction, Adventure, Family",150 min


In [31]:
# copie
df=res.copy()

In [32]:
# exportations en csv

df.to_csv('films.csv')

In [33]:
# exportations en excel
df.to_excel('films.xlsx')